## LSTM Model for parsed at sentence level text (Melisa)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:

np.random.seed(11)



import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset



In [9]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dense, Dropout, SpatialDropout1D, BatchNormalization
from tensorflow.keras.metrics import Precision, Recall
from sklearn.metrics import f1_score

In [11]:
import sys
sys.path.append('/content/drive/MyDrive/dataminingproje/source')
import lstm_final as lf


In [12]:
"""
train_path = "../data/train_preprocessed.json"
test_path = "../data/test.json"
embeddings_path = "../data/glove.6B.50d.txt"
"""

train_path = "/content/drive/MyDrive/dataminingproje/data/train_preprocessed.json"
test_path = "/content/drive/MyDrive/dataminingproje/data/test.json"
embeddings_path = "/content/drive/MyDrive/dataminingproje/data/glove.6B.50d.txt"

max_words = 4000
embedding_dim = 50

train, train_l, test, test_l, tokenizer, max_seq_length = lf.read_dataset(train_path, test_path, max_words)
print("Prepared Data")
embedding_matrix = lf.embeddings(embeddings_path, embedding_dim, tokenizer, max_words)
print("Prepared embeddings matrix")

Prepared Data
Prepared embeddings matrix


In [16]:
dataset = tf.data.Dataset.from_tensor_slices((train, train_l))

# Shuffle and batch the dataset
batch_size = 256
buffer_size = 5000  # Set a buffer size for shuffling
dataset = dataset.shuffle(buffer_size).batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)

In [17]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, weights=[embedding_matrix], input_length=max_seq_length, trainable=False))
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(BatchNormalization())
model.add(LSTM(32, return_sequences=True))
model.add(LSTM(16))
model.add(BatchNormalization())
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

#TODO: BIDIRECTIONAL VS LSTM DENE, BATCHNORMALİZATION OLMADAN DA DENE, DENSE LAYER ARTTIRMAYI DENE

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', Precision(), Recall()])
model.fit(train, train_l, epochs=6, batch_size=64)

In [23]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 2619, 50)          200000    
                                                                 
 bidirectional_1 (Bidirecti  (None, 2619, 128)         58880     
 onal)                                                           
                                                                 
 batch_normalization_2 (Bat  (None, 2619, 128)         512       
 chNormalization)                                                
                                                                 
 lstm_4 (LSTM)               (None, 2619, 32)          20608     
                                                                 
 lstm_5 (LSTM)               (None, 16)                3136      
                                                                 
 batch_normalization_3 (Bat  (None, 16)               

In [24]:
test_loss, test_accuracy, test_precision, test_recall = model.evaluate(test, test_l, verbose=0)

In [25]:
print(f"Test Accuracy: {test_accuracy}")
print(f"Test Precision: {test_precision}")
print(f"Test Recall: {test_recall}")
print(f"Test F1: {2*test_recall*test_precision / (test_recall+test_precision)}")

Test Accuracy: 0.7667297720909119
Test Precision: 0.7888904213905334
Test Recall: 0.43653225898742676
Test F1: 0.5620527892252423


In [ ]:
predictions = model.predict(test)

469/469 [==============================] - 53s 111ms/step


In [ ]:
model.save_weights("trained_model_lstm", save_format="h5")


In [ ]:
from tensorflow.keras import layers, Input, Model
# RANDOM FORESTA FEEDLEMEK İÇİN
def createLSTMModel():
    emb_input = Input(shape=(1,), name="emb")
    embedding = layers.Embedding(input_dim=4000, output_dim=50, input_length=2048,weights=[embedding_matrix], trainable=False)(emb_input)
    lstm1 = layers.LSTM(64, return_sequences=True)(embedding)
    lstm2 = layers.LSTM(16)(lstm1)
    out = layers.Dense(1, activation="sigmoid")(lstm2)
    model = Model(inputs=emb_input, outputs=out)
    return model

model = createLSTMModel()
model.load_weights("trained_model_lstm") # bana attığın train edilmiş model
pretrained_model = tf.keras.Model(
    inputs=model.inputs,
    outputs=model.layers[-2].output
)

X_test = padded_sequences  # train datası
preds = pretrained_model.predict(X_test)

ValueError: ignored

In [ ]:
preds_np = np.array(preds)
np.save('lstm-preds.npy', preds_np)